In [1]:
import pandas as pd

In [2]:
def sign(date_string, *, lat_long=(-25, -46), date_format='%d/%m/%Y'):
    from datetime import datetime

    from flatlib.datetime import Datetime
    from flatlib.const import SUN
    from flatlib.geopos import GeoPos
    from flatlib.chart import Chart
    
    return Chart(
        Datetime(
            datetime
            .strptime(date_string, date_format)
            .strftime('%Y/%m/%d')
        ),
        GeoPos(*lat_long),
    ).get(SUN).sign

In [3]:
candidatos = (
    pd
    .read_excel('./data/interim/candidatos_2018.xlsx')
    .loc[lambda df: (
        (df.DS_DETALHE_SITUACAO_CAND == 'DEFERIDO')
        | (df.DS_DETALHE_SITUACAO_CAND == 'DEFERIDO COM RECURSO')
    )]
    .assign(SIGNO=lambda df: df.DT_NASCIMENTO.map(sign))
)

candidatos.head()

,DT_GERACAO,HH_GERACAO,ANO_ELEICAO,CD_TIPO_ELEICAO,NM_TIPO_ELEICAO,NR_TURNO,CD_ELEICAO,DS_ELEICAO,DT_ELEICAO,TP_ABRANGENCIA,...,CD_OCUPACAO,DS_OCUPACAO,NR_DESPESA_MAX_CAMPANHA,CD_SIT_TOT_TURNO,DS_SIT_TOT_TURNO,ST_REELEICAO,ST_DECLARAR_BENS,NR_PROTOCOLO_CANDIDATURA,NR_PROCESSO,SIGNO
0,30/10/2018,10:39:17,2018,2,ELEIÇÃO ORDINÁRIA,1,297,Eleições Gerais Estaduais 2018,07/10/2018,ESTADUAL,...,257,EMPRESÁRIO,-1,5,SUPLENTE,N,S,-1,6004320720186009600,Taurus
1,30/10/2018,10:39:17,2018,2,ELEIÇÃO ORDINÁRIA,1,297,Eleições Gerais Estaduais 2018,07/10/2018,ESTADUAL,...,177,DETETIVE PARTICULAR,0,5,SUPLENTE,N,N,-1,6001601320186009600,Capricorn
2,30/10/2018,10:39:17,2018,2,ELEIÇÃO ORDINÁRIA,1,297,Eleições Gerais Estaduais 2018,07/10/2018,ESTADUAL,...,292,AGENTE ADMINISTRATIVO,0,5,SUPLENTE,N,N,-1,6003004720186009600,Cancer
4,30/10/2018,10:39:17,2018,2,ELEIÇÃO ORDINÁRIA,1,297,Eleições Gerais Estaduais 2018,07/10/2018,ESTADUAL,...,271,MAGISTRADO,-1,2,ELEITO POR QP,N,S,-1,6003974720186009600,Virgo
5,30/10/2018,10:39:17,2018,2,ELEIÇÃO ORDINÁRIA,1,297,Eleições Gerais Estaduais 2018,07/10/2018,ESTADUAL,...,277,DEPUTADO,0,2,ELEITO POR QP,S,S,-1,6002432920186009600,Aquarius


In [6]:
candidatos.groupby('SIGNO').size().sort_values(ascending=False)

SIGNO
Aries          2286
Taurus         2274
Gemini         2263
Libra          2254
Leo            2235
Virgo          2219
Cancer         2205
Pisces         2140
Scorpio        2104
Aquarius       2032
Capricorn      2021
Sagittarius    1983
dtype: int64

In [77]:
with pd.option_context('display.max_rows', None):
    display(
        candidatos
        .groupby([
            'DS_CARGO',
            'DS_SIT_TOT_TURNO',
            'SIGNO',
        ])
        .size()
        .rename('TOTAL')
        .unstack(0)
        .fillna('')
    )

DS_CARGO                     1º SUPLENTE 2º SUPLENTE DEPUTADO DISTRITAL  \
DS_SIT_TOT_TURNO SIGNO                                                    
2º TURNO         Aquarius                                                 
                 Aries                                                    
                 Cancer                                                   
                 Capricorn                                                
                 Gemini                                                   
                 Leo                                                      
                 Libra                                                    
                 Pisces                                                   
                 Sagittarius                                              
                 Scorpio                                                  
                 Taurus                                                   
ELEITO           Aquarius              3           3                      
                 Aries                 6           7                      
                 Cancer               10           4                      
                 Capricorn             2           3                      
                 Gemini                6           8                      
                 Leo                   3           3                      
                 Libra                 5           4                      
                 Pisces                6           6                      
                 Sagittarius           2           3                      
                 Scorpio               5           2                      
                 Taurus                3           6                      
                 Virgo                 3           5                      
ELEITO POR MÉDIA Aquarius                                                 
                 Aries                                                    
                 Cancer                                               1   
                 Capricorn                                            2   
                 Gemini                                               2   
                 Leo                                                  2   
                 Libra                                                    
                 Pisces                                                   
                 Sagittarius                                          1   
                 Scorpio                                              2   
                 Taurus                                                   
                 Virgo                                                1   
ELEITO POR QP    Aquarius                                                 
                 Aries                                                1   
                 Cancer                                               1   
                 Capricorn                                            1   
                 Gemini                                               1   
                 Leo                                                  2   
                 Libra                                                    
                 Pisces                                                   
                 Sagittarius                                          2   
                 Scorpio                                              2   
                 Taurus                                                   
                 Virgo                                                3   
NÃO ELEITO       Aquarius             21          17                 13   
                 Aries                16          21                 26   
                 Cancer               23          15                 11   
                 Capricorn            29          17                 19   
                 Gemini  

In [68]:
with pd.option_context('display.max_rows', None):
    display(
        candidatos
        .groupby(['DS_SIT_TOT_TURNO', 'SIGNO'])
        .size()
        .rename('TOTAL')
        .reset_index()
        .sort_values(['DS_SIT_TOT_TURNO', 'TOTAL'], ascending=[True, False])
        .set_index(['DS_SIT_TOT_TURNO', 'SIGNO'])
    )

TOTAL
DS_SIT_TOT_TURNO SIGNO             
2º TURNO         Cancer          10
                 Taurus          10
                 Leo              8
                 Pisces           8
                 Aquarius         6
                 Aries            5
                 Gemini           4
                 Scorpio          4
                 Libra            2
                 Sagittarius      2
                 Capricorn        1
ELEITO           Cancer          26
                 Aries           25
                 Taurus          25
                 Pisces          24
                 Gemini          22
                 Leo             18
                 Libra           16
                 Scorpio         16
                 Virgo           13
                 Aquarius        12
                 Capricorn       11
                 Sagittarius     10
ELEITO POR MÉDIA Gemini          30
                 Scorpio         30
                 Aries           29
                 Virgo           29
                 Cancer          28
                 Leo             28
                 Taurus          24
                 Aquarius        22
                 Libra           22
                 Pisces          21
                 Sagittarius     18
                 Capricorn       17
ELEITO POR QP    Gemini         116
                 Taurus         113
                 Leo            112
                 Aries          111
                 Cancer         110
                 Capricorn      110
                 Virgo          107
                 Pisces         105
                 Libra          103
                 Aquarius        99
                 Sagittarius     99
                 Scorpio         89
NÃO ELEITO       Taurus         501
                 Gemini         489
                 Leo            481
                 Libra          471
                 Virgo          466
                 Aquarius       463
                 Aries          460
                 Cancer         460
                 Capricorn      460
                 Scorpio        456
                 Pisces         454
                 Sagittarius    412
SUPLENTE         Aries         1656
                 Libra         1640
                 Virgo         1604
                 Gemini        1602
                 Taurus        1601
                 Leo           1588
                 Cancer        1571
                 Pisces        1528
                 Scorpio       1509
                 Sagittarius   1442
                 Aquarius      1430
                 Capricorn     1422

In [57]:
with pd.option_context('display.max_rows', None):
    display(
        candidatos
        .groupby([
            'SG_PARTIDO',
            'SIGNO',
        ])
        .size()
        .rename('TOTAL')
        .reset_index(['SIGNO'])
        .join(
            candidatos
            .groupby('SG_PARTIDO')
            .size()
            .rename('TOTAL_PARTIDO')
        )
        .assign(
            PROPORCAO=lambda df: df.TOTAL / df.TOTAL_PARTIDO
        )
        .reset_index()
        .loc[lambda df: df.groupby('SG_PARTIDO').PROPORCAO.idxmax()]
        .set_index('SG_PARTIDO')
    )

,SIGNO,TOTAL,TOTAL_PARTIDO,PROPORCAO
SG_PARTIDO,,,,
AVANTE,Gemini,95,934,0.101713
DC,Aries,70,647,0.108192
DEM,Capricorn,64,647,0.098918
MDB,Aquarius,96,1018,0.094303
NOVO,Scorpio,42,411,0.102190
PATRI,Taurus,112,1078,0.103896
PC do B,Virgo,79,754,0.104775
PCB,Cancer,15,80,0.187500
PCO,Taurus,6,29,0.206897
